## 네이버 영화 리뷰 감성분석 Ver.2
- tokenizer 함수
- TfidfVectozier + NaiveBayes

In [2]:
import pandas as pd
train_df = pd.read_csv('data/네이버영화리뷰 train 전처리완료.tsv', sep='\t')
test_df = pd.read_csv('data/네이버영화리뷰 test 전처리완료.tsv', sep='\t')

In [4]:
X_train, y_train = train_df.document.values, train_df.label.values
X_test, y_test = test_df.document.values, test_df.label.values

- Tokenizer 함수

In [5]:
with open('data/한글불용어100.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]

In [6]:
from konlpy.tag import Okt
okt = Okt()

In [7]:
def okt_tokenizer(text):
    morphs = okt.morphs(text, stem=True)
    tokens = [morph for morph in morphs if morph not in stop_words]
    return tokens

- Pipeline 학습

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [11]:
pipeline = Pipeline([
    ('tvect', TfidfVectorizer(tokenizer=okt_tokenizer, token_pattern=None)),
    ('nb', MultinomialNB())
])
%time pipeline.fit(X_train, y_train)

CPU times: total: 18min 56s
Wall time: 18min 37s


Pipeline(steps=[('tvect',
                 TfidfVectorizer(tokenizer=<function okt_tokenizer at 0x000002E255612660>)),
                ('nb', MultinomialNB())])

In [12]:
%time pipeline.score(X_test, y_test)

CPU times: total: 6min 16s
Wall time: 6min 10s


0.8356464423155654

- 실제 데이터 적용

In [13]:
reviews = [
    '이 영화 개꿀잼 ㅋㅋㅋ', '이 영화 핵노잼 ㅠㅠ', '이딴게 영화냐 ㅉㅉ',
    '감독 뭐하는 놈이냐?', '와 개쩐다 정말 세계관 최강자들의 영화다'
]

In [14]:
# 전처리 - 한글 이외의 문자는 공백으로 변환
import re
reviews = list(map(lambda s: re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', s), reviews))
reviews

['이 영화 개꿀잼 ㅋㅋㅋ',
 '이 영화 핵노잼 ㅠㅠ',
 '이딴게 영화냐 ㅉㅉ',
 '감독 뭐하는 놈이냐 ',
 '와 개쩐다 정말 세계관 최강자들의 영화다']

In [15]:
pipeline.predict(reviews)

array([1, 0, 0, 0, 1], dtype=int64)

- 최적 파라메터 찾기
    - tokenizer를 설정하지 말고, 형태소 분석이 끝난 데이터로 수행할 것

- 모델 저장하기

In [16]:
import joblib
joblib.dump(pipeline, 'model/네이버영화리뷰 pipeline.pkl')

['model/네이버영화리뷰 pipeline.pkl']